# Overview: Plate tectonic reconstructions and porphyry copper deposit formation in the Andes

This notebook demonstrates a workflow for reconstructing the subduction dynamics along the Andean margin and investigating their relationship with the timing and location of porphyry copper deposit formation. Using the **gplately** Python package and related geospatial libraries, the notebook guides the user through:

- Importing and filtering a geospatial dataset of Andean porphyry copper deposits.
- Visualizing deposit locations and ages within a defined South American extent.
- Initializing a plate reconstruction model (Muller2019) and anchoring reconstructions to the South American plate.
- Reconstructing deposit locations through geological time.
- Extracting and visualizing subduction zone properties (e.g., convergence rates, seafloor age, carbonate sediment thickness) along the trench as functions of latitude and time.
- Interpolating subduction parameters to the birth age and location of each deposit.
- Analyzing relationships and correlations between subduction zone properties and deposit formation, including clustering analysis.


The notebook provides a reproducible approach to examine how variations in subduction flux, seafloor age, and sediment flux may have influenced the formation of porphyry copper deposits in the Andes. This example is fairly straightforward compared to the analysis outlined in Mather et al. (2025) and related references.




-----
## 1. Setup and Package Import

**Purpose**  
Load all Python modules needed for geospatial data handling, plotting, and plate reconstructions.

**Key imports**  
- `gplately`: tools for plate reconstructions and tectonic visualization.  
- `PlateModelManager`: interface to retrieve rotation models and topologies.  
- `numpy`, `pandas`: numeric and table data handling.  
- `matplotlib`, `geopandas`: plotting and geospatial data structures.

In [ ]:
# This cell is only required if we're running in Google Colab
! pip install pandas geopandas numpy matplotlib seaborn scikit-learn cartopy scipy gplately


In [ ]:
import gplately
from plate_model_manager import PlateModelManager
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
from sklearn.cluster import KMeans

import cartopy.crs as ccrs
import cartopy.feature as cfeature

from scipy.ndimage import uniform_filter1d
from scipy.interpolate import griddata
from scipy.stats import pearsonr


---
## 2. Import and Prepare Deposit Data

**Purpose**  
Read the porphyry copper deposit dataset and filter for entries that include age information.

**What happens here**  
1. Load a geodatabase file containing deposit locations and ages.  
2. Remove any records where age (`Min_Age_Ma`) is missing, since subsequent analysis requires a valid age.

In [ ]:
# Load deposit data from a geodatabase file.
Andes_porphyry_deposits = gpd.read_file("https://github.com/TarynScharf/PythonWorkshop/raw/95907ef48eb87d25be1fb8acc349ab0c07b86fe7/SEG/Data_for_exercises/Andes%20Porphyry%20Copper.gdb")

# Keep only rows where 'Min_Age_Ma' is not null
Andes_porphyry_deposits.dropna(inplace=True, subset=['Min_Age_Ma'])

---


## 3. Define Geographic Extent

**Purpose**  
Specify the bounding box for South America so that maps and reconstructions focus on the region of interest.

**Details**  
- The extent is `[west_lon, east_lon, south_lat, north_lat]`.  
- Coordinates are in degrees.

In [ ]:
SA_extent = [-90, -50, -60, 20]

---

## 4. Visualize Deposits on a Map

**Purpose**  
Create a map of porphyry deposit locations, colored by their formation age.

**Steps**  
1. Initialize a Cartopy map with PlateCarree projection.  
2. Add land, ocean, coastline, borders, lakes, and rivers for context.  
3. Plot each deposit as a colored marker where the color scale represents age in million years.

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent(SA_extent, crs=ccrs.PlateCarree())

# Add map features for context.
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)

# Plot the deposits, colored by age.
Andes_porphyry_deposits.plot(
    column='Min_Age_Ma', ax=ax, cmap='viridis', markersize=20, edgecolor='k',
    legend=True, legend_kwds={'label': "Deposit Age (Ma)"}, vmin=0, vmax=100,
    transform=ccrs.PlateCarree()
)

*Result*: You should see a scatter of deposit points over South America, coloured by age of deposit

---

## 5. Plate Reconstruction Setup

**Purpose**  
Initialize the plate reconstruction model and anchor South America so
all reconstructions are relative to its fixed position.

**Details**  
- We fetch the `Muller2019` plate rotation model.  
- We fix plate ID 202 (South America) in place.  
- We prepare topology layers for coastlines, continental outlines, and
conjugate oceanic basins.

In [ ]:
pm_manager = PlateModelManager()
plate_model = pm_manager.get_model("Muller2019", data_dir="plate-model-repo")

# Fix the South American plate in place for reconstructions.
anchor_plate_id = 202

model = gplately.PlateReconstruction(
    plate_model.get_rotation_model(),
    plate_model.get_topologies(),
    plate_model.get_static_polygons(),
    anchor_plate_id=anchor_plate_id
)

# Set up plotting for reconstructed tectonic features.
gplot = gplately.plot.PlotTopologies(
    model,
    plate_model.get_layer('Coastlines'),
    plate_model.get_layer('ContinentalPolygons'),
    plate_model.get_layer('COBs')
)

---

## 6. Add Deposits as Reconstructable Points

**Purpose**  
Convert deposit coordinates into a `gplately.Points` object so we can reconstruct their positions through geological time.

**Key inputs**  
- Longitude and latitude arrays from the deposit GeoDataFrame.  
- Birth ages of each deposit (`Min_Age_Ma`).  
- The same anchor plate ID to keep South America fixed.

In [ ]:
gpts = gplately.Points(
    model,
    Andes_porphyry_deposits.Longtitude.values,
    Andes_porphyry_deposits.Latitude.values,
    age=Andes_porphyry_deposits.Min_Age_Ma.values,
    anchor_plate_id=anchor_plate_id
)

---

## 7. Visualization of Plate Configurations

**Purpose**  
Plot reconstructed coastlines, trenches, and deposit points at a chosen time slice (100 Ma in this example).

**Steps**  
1. Set `gplot.time` to the desired reconstruction age.  
2. Draw coastlines, subduction trenches, and subduction teeth.  
3. Reconstruct deposit point positions and plot with age coloring.

In [ ]:
time_ma = 100
gplot.time = time_ma

fig = plt.figure(figsize=(16,12))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
gplot.plot_coastlines(ax, color='0.5')
gplot.plot_trenches(ax, color='k')
gplot.plot_subduction_teeth(ax, color='k')

long, lat, ind = gpts.reconstruct(time_ma, return_array=True, return_point_indices=True, anchor_plate_id=anchor_plate_id)
cmap = ax.scatter(long, lat, cmap='viridis', c=gpts.age[ind], s=15, transform=ccrs.PlateCarree(), vmin=0, vmax=200)
plt.colorbar(cmap, ax=ax, label='Deposit Age (Ma)', shrink=0.5)

*This plot shows the reconstructed positions of deposits and tectonic features at 100 million years ago.*

---

## 8. Extract Subduction Zone Data Function

**Purpose**  
Define a helper function to sample subduction trench properties (convergence rate, trench velocity, etc.) within the South American extent at any given time.

**Function details**  
- Calls `model.tessellate_subduction_zones` to retrieve data at trench.
- Filters by longitude/latitude box (`SA_extent`).  
- Returns a pandas DataFrame with columns:  
- `lon`, `lat`: sample position  
- `conv_rate`, `conv_angle`: convergence magnitude and obliquity  
- `trench_velocity`, `trench_velocity_angle`: trench motion relative to
anchor plate  
- `arc_length`, `trench_azimuth_angle`, `subducting_pid`, `trench_pid`

In [ ]:
lat_samples = np.arange(-55, 20, 0.1)

def get_south_america_subduction_zone_data(r12n_time):
    """
    Extract subduction zone data for South America at a given time.
    Filters for points within the South American region.
    """
    subduction_data = model.tessellate_subduction_zones(
        r12n_time,
        tessellation_threshold_radians=0.01,
        anchor_plate_id=anchor_plate_id
    )
    ret = []
    for row in subduction_data:
        # Keep only points within the South American extent.
        if row[0] > SA_extent[0] and row[0] < SA_extent[1] and row[1] > SA_extent[2] and row[1] < SA_extent[3]:
            ret.append(row)
    ret.sort(key=lambda row: row[1])
    ret_df = pd.DataFrame(ret, columns=['lon', 'lat', 'conv_rate', 'conv_angle', 'trench_velocity', 'trench_velocity_angle', 'arc_length',
                                        'trench_azimuth_angle', 'subducting_pid', 'trench_pid'])
    return ret_df[(ret_df.lat > lat_samples.min()) & (ret_df.lat < lat_samples.max())]

### 8a. Visualize Subduction Data at a Single Time

**Purpose**  
Use our helper function to plot convergence velocity samples at 50 Ma on a Mollweide projection.

In [ ]:
reconstruction_time = 50
# Retrieve subduction data at 50 Ma
south_america_subduction_data = get_south_america_subduction_zone_data(reconstruction_time)

fig = plt.figure(figsize=(10,6), dpi=100)
ax = fig.add_subplot(111, projection=ccrs.Mollweide(central_longitude = 0))



gl=ax.gridlines(color='0.7',linestyle='--', xlocs=np.arange(-180,180,15), ylocs=np.arange(-90,90,15))
gl.left_labels = True

plt.title(f'{reconstruction_time} Ma')
# you may change the extent to global to see the sample points in a world map.
#ax.set_global()
ax.set_extent(SA_extent, crs=ccrs.PlateCarree())

gplot.time = reconstruction_time
gplot.plot_coastlines(ax, color='grey')

# Latitudes and longitudes of points along trench segments
subduction_lon = south_america_subduction_data.lon
subduction_lat = south_america_subduction_data.lat

conv_rate = south_america_subduction_data.conv_rate # subducting convergence (relative to trench) velocity magnitude (in cm/yr)
cb=ax.scatter(subduction_lon,subduction_lat, marker=".", s=5, c=conv_rate, transform=ccrs.PlateCarree(), cmap="inferno_r")

cbar = plt.colorbar(cb)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Convergence Velocity Magnitude (in cm/yr)', rotation=90)
plt.show()


### 8b. Compute Subduction Flux and Sample Grids Over Time

**Purpose**  
Loop from 0 to 80 Ma to compute:
- Area flux: trench-orthogonal convergence rate × arc length. 
- Volume flux (approximate) using a square-root age scaling. 
- Gridded arrays of convergence rate, seafloor age, and carbonate thickness for interpolation.

In [ ]:
# We are only interested in the reconstruction time from 80Ma to 0Ma
time_series = np.arange(0,81,1)

area_flux_series=[]
volume_flux_series=[]
age_array = []
conv_array = []
carbonate_array = []

for time in time_series:
    sa_sub_data = get_south_america_subduction_zone_data(time)
    conv_rate = sa_sub_data.conv_rate # subducting convergence (relative to trench) velocity magnitude (in cm/yr)
    conv_obliq = sa_sub_data.conv_angle # subducting convergence velocity obliquity angle (angle between trench normal vector and convergence velocity vector)
    arc_length = sa_sub_data.arc_length # length of arc segment (in degrees) that current point is on
    # Latitudes and longitudes of points along trench segments
    subduction_lon = sa_sub_data.lon
    subduction_lat = sa_sub_data.lat

    ortho_conv_rate = conv_rate*np.abs(np.cos(np.radians(conv_obliq)))
    area_flux = ortho_conv_rate * 100. * arc_length * 2 * np.pi * 6371000. / 360. # formula copied from Simon's Notebook

    #
    # Query age grid rasters
    #
    age_grid_raster = gplately.Raster(
        data=plate_model.get_raster("AgeGrids",time),
        plate_reconstruction=model,
        extent=SA_extent,
    )
    age_grid_raster=age_grid_raster.clip_by_extent(SA_extent)
    ages = age_grid_raster.query(subduction_lon, subduction_lat,region_of_interest=200)

    #
    # Query carbonate sediment thickness rasters
    #
    carbonate_thickness_raster = gplately.Raster(
        data=plate_model.get_raster("SedimentThickness",time),
        plate_reconstruction=model,
        extent=SA_extent,
    )
    carbonate_thickness_raster=carbonate_thickness_raster.clip_by_extent(SA_extent)
    carbonate_thickness = carbonate_thickness_raster.query(subduction_lon, subduction_lat,region_of_interest=200)
    
    #
    # we have two methods to get the volume flux
    #
    # method 1: use gplately.tools.plate_isotherm_depth
    # plate_depth = [gplately.tools.plate_isotherm_depth(seafloor_age) for seafloor_age in ages]
    # volume_flux = area_flux * plate_depth 
    
    # method 2: use Simon's formula
    volume_flux = area_flux * np.sqrt(ages) * 10.  # UNITS????

    area_flux_series.append(area_flux.sum())
    volume_flux_series.append(np.nansum(volume_flux))

    conv_array.append(np.interp(lat_samples, subduction_lat, ortho_conv_rate, left=np.nan, right=np.nan))
    age_array.append(np.interp(lat_samples, subduction_lat, ages, left=np.nan, right=np.nan))
    carbonate_array.append(np.interp(lat_samples, subduction_lat, carbonate_thickness, left=np.nan, right=np.nan))
    
    gplately.tools.update_progress(time/time_series.size)  
gplately.tools.update_progress(1)

---

## 9. Analyze and Visualize Subduction Properties

With our computed time series and gridded samples, we can now create plots that reveal how subduction flux, convergence rate, seafloor age, and carbonate thickness varied through time and latitude.

### 9a. Prepare Deposit Birth Reconstructions

**Purpose**  
Get each deposit’s reconstructed birth coordinates (longitude, latitude) and age for overlaying on our time–latitude diagrams.

In [ ]:
points_loglat_at_birth = np.array(gpts.reconstruct_to_birth_age(ages=gpts.age))
points_age = gpts.age

### 9b. Plot Reconstructed Subduction Flux Time Series

**Purpose**  
Compare total area flux and volume flux through time, and mark deposit formation ages.
-   Left axis: area flux (m²/yr)  
-   Right axis: volume flux (m³/yr)  
-   Vertical markers: deposit ages

In [ ]:
from scipy.ndimage import uniform_filter1d

fig = plt.figure(figsize=(8,4), dpi=100)
ax = fig.add_subplot(111)

N=3
ax.plot(time_series, uniform_filter1d(area_flux_series, size=N), 'b', label='area flux')
ax.set_xlim(80,0)
ax.set_ylim(1e9,11e9)
ax.grid(axis='x', linestyle=':')
#ax.set_xticklabels([])
ax.set_ylabel('Area Flux', fontsize=14, color='b')
ax.tick_params(axis='y', labelcolor='b')

ax1 = ax.twinx() 
ax1.plot(time_series, uniform_filter1d(np.array(volume_flux_series), size=N), 'r', label='volume flux')
ax.set_xlabel('Reconstruction Time [Ma]', fontsize=14)
ax1.set_ylabel('Volume Flux', fontsize=14, color='r')
ax1.set_ylim(0e11,7e11)
ax1.tick_params(axis='y', labelcolor='r')
#ax.legend()
ax.set_title('Reconstructed Subduction Flux', fontsize=16)

### plot formation age to see if there is any correlation
ax.plot(Andes_porphyry_deposits['Min_Age_Ma'], np.full(Andes_porphyry_deposits['Min_Age_Ma'].shape, 1e9), 'k|', markersize=20, label='Deposit Ages')

plt.show()




### 9c. Visualize Trench-Orthogonal Convergence Rate

**Purpose**  
Display how the orthogonal component of convergence varied with latitude and time, and overlay deposit birth points.

In [ ]:
fig = plt.figure(figsize=(8,4), dpi=100)
ax = fig.add_subplot(111)

m = ax.pcolormesh(time_series, 
               lat_samples, 
               np.array(conv_array).T, 
               vmin=5, vmax=15, cmap='Oranges')

fig.colorbar(m, orientation="vertical", extend='max', shrink=0.9, pad=0.03, label='Convergence Rate [cm/yr]')

ax.scatter(points_age, points_loglat_at_birth.T[:,1], c='k', s=5, alpha=0.9)

ax.set_xlim(80,0)
ax.set_ylim(-50,5)
ax.set_xlabel('Reconstruction Time [Ma]', fontsize=14)
ax.set_ylabel('Latitude', fontsize=14)
ax.set_title('Trench-Orthogonal Convergence Rate', fontsize=16)
plt.show()



### 9d. Visualize Subducting Seafloor Age

**Purpose**  
Show the age of the subducting seafloor along the trench as a function of time and latitude.

In [ ]:
fig = plt.figure(figsize=(8,4), dpi=100)
ax = fig.add_subplot(111)

m = ax.contourf(time_series, 
               lat_samples, 
               np.array(age_array).T, 
               levels=np.arange(0,101,5), cmap='plasma_r', extend='max')

fig.colorbar(m, orientation="vertical", extend='max', shrink=0.9, pad=0.03, label='Seafloor Age [Myr]')

ax.scatter(points_age, points_loglat_at_birth.T[:,1], c='k', s=5, alpha=0.9)

ax.set_xlim(80,0)
ax.set_ylim(-50,5)
ax.set_xlabel('Reconstruction Time [Ma]', fontsize=14)
ax.set_ylabel('Latitude', fontsize=14)
ax.set_title('Subducting Seafloor Age', fontsize=16)
plt.show()



### 9e. Visualize Carbonate Sediment Thickness

**Purpose**  
Illustrate how carbonate sediment thickness on the subducting plate varied through time and latitude.

In [ ]:
fig = plt.figure(figsize=(8,4), dpi=100)
ax = fig.add_subplot(111)

m = ax.contourf(time_series, 
               lat_samples, 
               np.array(carbonate_array).T, 
               levels=[0,25,50,75,100,125,150, 175, 200], vmax=200, cmap='GnBu', extend='max')

fig.colorbar(m, orientation="vertical", extend='max', shrink=0.9, pad=0.03, label='Carbonate Sediment Thickness [meter]')

ax.scatter(points_age, points_loglat_at_birth.T[:,1], c='k', s=5, alpha=0.9)

ax.set_xlim(80,0)
ax.set_ylim(-50,5)
ax.set_xlabel('Reconstruction Time [Ma]', fontsize=14)
ax.set_ylabel('Latitude', fontsize=14)
ax.set_title('Thickness of Subducting Carbonate Sediment', fontsize=16)
plt.show()



---

## 10. Relate Subduction Properties to Deposit Formation

**Purpose**  
Interpolate subduction parameters (seafloor age, carbonate thickness) to the birth location and age of each deposit, then analyze relationships..

### 10a. Reconstruct Deposit Locations at Birth Age

**Purpose**  
Obtain the birth coordinates (lon, lat) and ages for each deposit from our `gpts` object.

In [ ]:
#### get the seafloor age or thickness of sediments at the porphyry deposit locations
points_loglat_at_birth = np.array(gpts.reconstruct_to_birth_age(ages=gpts.age))
points_age = gpts.age
points_lon = points_loglat_at_birth.T[:,0]
points_lat = points_loglat_at_birth.T[:,1]

### 10b. Interpolate Subduction Parameters at Deposit Points

**Purpose**  
Use our precomputed grids to interpolate seafloor age and carbonate thickness onto each deposit’s birth coordinates and age.

In [ ]:
### we use the grids from above to interpolate the seafloor age and carbonate thickness at the deposit locations

seafloor_age_at_deposits = griddata((lat_samples.repeat(time_series.size), np.tile(time_series, lat_samples.size)), np.array(age_array).flatten(), (points_lat, points_age), method='linear')

carbonate_thickness_at_deposits = griddata((lat_samples.repeat(time_series.size), np.tile(time_series, lat_samples.size)), np.array(carbonate_array).flatten(), (points_lat, points_age), method='linear')

Andes_porphyry_deposits['Seafloor_Age_at_Deposit'] = seafloor_age_at_deposits
Andes_porphyry_deposits['Carbonate_Thickness_at_Deposit'] = carbonate_thickness_at_deposits

### drop any rows with NaN values in either of the two new columns
Andes_porphyry_deposits.dropna(inplace=True, subset=['Seafloor_Age_at_Deposit', 'Carbonate_Thickness_at_Deposit'])

### 10c. Map Interpolated Parameters

**Purpose**  
Plot two maps side by side showing seafloor age and carbonate thickness at each deposit’s birth point on a present-day basemap.

In [ ]:
gplot.time = 0

### plot seafloor age and carbonate thickness at deposit locations
fig, ax = plt.subplots(1, 2, figsize=(8,6), dpi=100, subplot_kw={'projection': ccrs.PlateCarree()})
cbar = Andes_porphyry_deposits.plot(column='Seafloor_Age_at_Deposit', ax=ax[0], cmap='plasma_r', markersize=50, edgecolor='k', legend=True, legend_kwds={'label': "Seafloor Age at Deposit (Ma)"}, vmin=0, vmax=100)

gplot.plot_coastlines(ax[0], color='0.5')
gplot.plot_trenches(ax[0], color='k')
gplot.plot_subduction_teeth(ax[0], color='k')


ax[0].set_title('Seafloor Age', fontsize=10,)

ax[0].set_extent(SA_extent, crs=ccrs.PlateCarree())

cbar1 = Andes_porphyry_deposits.plot(column='Carbonate_Thickness_at_Deposit', ax=ax[1], cmap='GnBu', markersize=50, edgecolor='k', legend=True, legend_kwds={'label': "Carbonate Sediment Thickness at Deposit (m)"}, vmin=0, vmax=200)
ax[1].set_title('Carbonate Sediment Thickness', fontsize=10)

gplot.plot_coastlines(ax[1], color='0.5')
gplot.plot_trenches(ax[1], color='k')
gplot.plot_subduction_teeth(ax[1], color='k')

ax[1].set_extent(SA_extent, crs=ccrs.PlateCarree())

plt.show()

### 10d. Scatter Plot and Correlation Analysis

**Purpose**  
Visualize the relationship between seafloor age and carbonate sediment thickness at deposit formation, and calculate the Pearson correlation coefficient.

In [ ]:
plt.scatter(Andes_porphyry_deposits.Min_Age_Ma, Andes_porphyry_deposits.Carbonate_Thickness_at_Deposit)
plt.xlabel('Deposit age (Ma)')
plt.ylabel('Carbonate Sediment Thickness at deposit formation (m)')
plt.grid()
### get the correlation coefficient
corrCoeff = np.corrcoef(Andes_porphyry_deposits.Min_Age_Ma, Andes_porphyry_deposits.Carbonate_Thickness_at_Deposit, rowvar=False)

print(corrCoeff)


In [ ]:
plt.scatter(Andes_porphyry_deposits.Min_Age_Ma, Andes_porphyry_deposits.Seafloor_Age_at_Deposit)
plt.xlabel('Deposit age (Ma)')
plt.ylabel('Seafloor Age at deposit formation (Ma)')
plt.grid()
### get the correlation coefficient
corrCoeff = np.corrcoef(Andes_porphyry_deposits.Min_Age_Ma, Andes_porphyry_deposits.Seafloor_Age_at_Deposit, rowvar=False)

print(corrCoeff)


In [ ]:
plt.scatter(Andes_porphyry_deposits.Seafloor_Age_at_Deposit, Andes_porphyry_deposits.Carbonate_Thickness_at_Deposit)
plt.xlabel('Seafloor Age at deposit formation (Ma)')
plt.ylabel('Carbonate Sediment Thickness at deposit formation (m)')
plt.grid()
### get the correlation coefficient
corrCoeff = np.corrcoef(Andes_porphyry_deposits.Seafloor_Age_at_Deposit, Andes_porphyry_deposits.Carbonate_Thickness_at_Deposit, rowvar=False)

print(corrCoeff)




### 10e. Cluster Analysis via K-Means

**Purpose**  
Identify potential clusters among deposits based on seafloor age and carbonate thickness using K-means clustering (k=2).

In [ ]:
### K-mean clustering to see if there are any clusters
kmeans = KMeans(n_clusters=2, random_state=1)
X = Andes_porphyry_deposits[['Seafloor_Age_at_Deposit', 'Carbonate_Thickness_at_Deposit']].values
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
Andes_porphyry_deposits

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
cbar = plt.scatter(X[:, 0], X[:, 1], c=Andes_porphyry_deposits.Min_Age_Ma, s=20, )
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.75, marker='X');
plt.xlabel('Seafloor Age at deposit formation (Ma)')
plt.ylabel('Carbonate Sediment Thickness at deposit formation (m)')
plt.colorbar(cbar, label='Deposit Age (Ma)', shrink=0.9)
plt.grid()
plt.show()



References:
-  Qin, X., Müller, R. D., Cannon, J., Landgrebe, T. C. W., Heine, C., Watson, R. J., and Turner, M. (2012) The GPlates Geological Information Model and Markup Language, Geosci. Instrum. Method. Data Syst., 1, 111–134, https://doi.org/10.5194/gi-1-111-2012.
- Mather, B.R., Müller, R.D., Zahirovic, S., Cannon, J., Chin, M., Ilano, L., Wright, N.M., Alfonso, C., Williams, S., Tetley, M., Merdith, A. (2023) Deep time spatio-temporal data analysis using pyGPlates with PlateTectonicTools and GPlately. Geoscience Data Journal, 1–8. https://doi.org/10.1002/gdj3.185
- Mather, B.R., Müller, R.D., Alfonso, C.P., Wright, N.M., Seton, M. (2025). Subducting seafloor anomalies promote porphyry copper formation. International Geology Review 1–17. https://doi.org/10.1080/00206814.2025.2508791
